# APPLE Stock Article extractions

In [1]:
import requests

# api_key = '23f5208b7e2940aeb8b72c67195565b2'
api_key = 'bd9bb272078b41ebb0ee0a905d4a9532'
date_from = '2023-12-09'
date_to = '2024-01-08'
# Expanded list of sources
sources = 'bloomberg,cnbc,reuters,financial-times,techcrunch,the-wall-street-journal,the-verge,business-insider,the-economist,wired,engadget,bbc-news,fortune,techradar'

# Broadening the search query
query = 'Apple OR "Apple Inc." OR "iPhone" OR "iPad" OR "Mac" OR "Apple Watch" OR "iOS" OR "App Store" OR "MacOS" OR "Apple Music" OR "Tim Cook"'

url = f'https://newsapi.org/v2/everything?q={query}&from={date_from}&to={date_to}&sources={sources}&apiKey={api_key}'
response = requests.get(url)
data = response.json()

# Check for 'articles' in response and extract headlines with publication dates
if 'articles' in data:
    news_data = [(article['title'], article['publishedAt'][:10]) for article in data['articles']]
    for headline, date in news_data:
        print(f"Date: {date}, Headline: {headline}")
else:
    print("Error or no articles found. Response:")
    print(data)


Date: 2024-01-06, Headline: 10 Best Deals on Laptops, Routers, and Apple Watch
Date: 2023-12-18, Headline: Apple will pause Watch Series 9 and Ultra 2 sales in the US due to a patent dispute
Date: 2023-12-31, Headline: Google Fixes Nearly 100 Android Security Issues
Date: 2023-12-21, Headline: Beeper says it's done playing cat and mouse with Apple over its iMessage for Android app
Date: 2024-01-01, Headline: Apple serial number lookup: How to identify your device
Date: 2023-12-26, Headline: Apple earnings: What investors should know about recent revenues, profits
Date: 2023-12-12, Headline: Google’s App Store Monopoly Ruled Illegal as Jury Sides With Epic
Date: 2024-01-03, Headline: 10 Best Deals on Fitness Trackers and Smartwatches
Date: 2024-01-07, Headline: iOS Explained: Apple's operating system version history, features, and iPhone capabilities
Date: 2023-12-25, Headline: Apple Developer Program offers tools to develop, distribute apps -- including forums, support
Date: 2023-12-11

# Sentiment Analysis on News Articles

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

# Extract just the headlines from news_data
headlines = [headline for headline, _ in news_data]

# Tokenize and encode headlines
max_length = 512  # You can adjust this based on your requirements
inputs = tokenizer(headlines, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Predict sentiment
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Pair each sentiment score with its corresponding date
sentiment_data = []
for (headline, date), prediction in zip(news_data, predictions):
    sentiment_data.append({
        'date': date,
        'headline': headline,
        'sentiment_positive': prediction[0].item(),
        'sentiment_neutral': prediction[1].item(),
        'sentiment_negative': prediction[2].item()
    })


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [3]:
sentiment_data

[{'date': '2024-01-06',
  'headline': '10 Best Deals on Laptops, Routers, and Apple Watch',
  'sentiment_positive': 0.5656136870384216,
  'sentiment_neutral': 0.4343480169773102,
  'sentiment_negative': 3.831055437331088e-05},
 {'date': '2023-12-18',
  'headline': 'Apple will pause Watch Series 9 and Ultra 2 sales in the US due to a patent dispute',
  'sentiment_positive': 0.00021442897559609264,
  'sentiment_neutral': 2.4236494482465787e-06,
  'sentiment_negative': 0.9997832179069519},
 {'date': '2023-12-31',
  'headline': 'Google Fixes Nearly 100 Android Security Issues',
  'sentiment_positive': 0.13971535861492157,
  'sentiment_neutral': 0.20315328240394592,
  'sentiment_negative': 0.6571313738822937},
 {'date': '2023-12-21',
  'headline': "Beeper says it's done playing cat and mouse with Apple over its iMessage for Android app",
  'sentiment_positive': 0.9999825954437256,
  'sentiment_neutral': 7.0791497819300275e-06,
  'sentiment_negative': 1.0406599358248059e-05},
 {'date': '2024

# Historical Stock Data and Sentiment Analysis Combined

In [6]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

# Convert sentiment_data to a DataFrame
sentiment_df = pd.DataFrame(sentiment_data)

# Convert date to datetime and set as index
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df.set_index('date', inplace=True)

# Aggregate sentiment scores by date
average_sentiment = sentiment_df.groupby('date').mean()

# Download stock data
ticker_symbol = 'AAPL'
stock_data = yf.download(ticker_symbol, start='2023-12-09', end='2024-01-08')

# Selecting the required columns
stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

# Convert stock_data index to datetime
stock_data.index = pd.to_datetime(stock_data.index)

# Combine stock data with sentiment data
combined_data = stock_data.join(average_sentiment)

# Interpolate missing values
combined_data = combined_data.interpolate(method='time')

# Scale price-related columns
price_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close']
scaler_prices = StandardScaler()
combined_data[price_cols] = scaler_prices.fit_transform(combined_data[price_cols])

# Scale Volume column independently
scaler_volume = StandardScaler()
combined_data['Volume'] = scaler_volume.fit_transform(combined_data[['Volume']])

print(combined_data)

[*********************100%%**********************]  1 of 1 completed
                Open      High       Low     Close  Adj Close    Volume  \
Date                                                                      
2023-12-11  0.100843 -0.019812  0.061040  0.153873   0.153873  0.143377   
2023-12-12  0.094840  0.226629  0.119483  0.446536   0.446536 -0.232229   
2023-12-13  0.497099  0.883806  0.729236  1.068200   1.068200  0.574262   
2023-12-14  1.083481  1.208387  0.984436  1.096891   1.096891  0.411545   
2023-12-15  0.985417  0.963949  1.148075  0.993600   0.993600  3.209194   
2023-12-18  0.697229  0.609315  0.639623  0.672246   0.672246 -0.093087   
2023-12-19  0.711240  0.673429  0.931836  0.873092   0.873092 -0.777994   
2023-12-20  0.859334  0.819690  0.725339  0.469489   0.469489 -0.252912   
2023-12-21  0.699233  0.699476  0.466243  0.440795   0.440795 -0.515269   
2023-12-22  0.515110  0.364877  0.362994  0.234214   0.234214 -0.941563   
2023-12-26  0.200908  0.060330 

<ipython-input-6-92393b1c0605>:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_sentiment = sentiment_df.groupby('date').mean()


In [7]:
import pandas as pd
df = pd.DataFrame(combined_data)
df

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2023-12-11,0.100843,-0.019812,0.061040,0.153873,0.153873,0.143377,0.741691,0.008316,0.249994
2023-12-12,0.094840,0.226629,0.119483,0.446536,0.446536,-0.232229,0.585718,0.140319,0.273963
2023-12-13,0.497099,0.883806,0.729236,1.068200,1.068200,0.574262,0.542164,0.318759,0.139077
2023-12-14,1.083481,1.208387,0.984436,1.096891,1.096891,0.411545,0.922950,0.000005,0.077046
2023-12-15,0.985417,0.963949,1.148075,0.993600,0.993600,3.209194,0.998695,0.000135,0.001170
2023-12-18,0.697229,0.609315,0.639623,0.672246,0.672246,-0.093087,0.663938,0.000104,0.335958
2023-12-19,0.711240,0.673429,0.931836,0.873092,0.873092,-0.777994,0.499856,0.000023,0.500121
2023-12-20,0.859334,0.819690,0.725339,0.469489,0.469489,-0.252912,0.500185,0.172500,0.327315
2023-12-21,0.699233,0.699476,0.466243,0.440795,0.440795,-0.515269,0.892206,0.000725,0.107070


# Transformer Architecture

In [48]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from tensorflow.keras.callbacks import EarlyStopping

selected_features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sentiment_positive', 'sentiment_neutral', 'sentiment_negative']
data = combined_data[selected_features]

train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:]

# Prepare features and target for model training
X_train = train_data.drop('Close', axis=1)
y_train = train_data['Close']
X_val = val_data.drop('Close', axis=1)
y_val = val_data['Close']
X_test = test_data.drop('Close', axis=1)
y_test = test_data['Close']

# Define and compile the neural network model
model = Sequential()
model.add(Dense(32, activation='LeakyReLU', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='LeakyReLU'))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train, y_train,
    epochs=50,  # Set back to 50 or an arbitrarily large number
    batch_size=8,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]  # Add the early stopping callback here
)

train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

# Convert the epochs range object to a list
epochs_list = list(epochs)

# Plotting the training and validation loss
loss_fig = go.Figure()
loss_fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name='Training Loss'))
loss_fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name='Validation Loss'))
loss_fig.update_layout(title='Training and Validation Loss per Epoch',
                       xaxis_title='Epochs',
                       yaxis_title='Loss',
                       legend_title='Type')
loss_fig.show()

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'MSE: {mse}, MAE: {mae}, R^2: {r2}')

# Reshape y_test and predictions for inverse transform
y_test_reshaped = y_test.values.reshape(-1, 1)
predictions_reshaped = predictions.reshape(-1, 1)

num_scaled_cols = 5

# Create separate dummy arrays for inverse scaling
dummy_array_y_test = np.zeros((len(y_test_reshaped), num_scaled_cols))
dummy_array_predictions = np.zeros((len(predictions_reshaped), num_scaled_cols))

# Fill in the 'Close' column values in the dummy arrays
# Assuming 'Close' is the last of the scaled columns
dummy_array_y_test[:, -1] = y_test_reshaped.flatten()
dummy_array_predictions[:, -1] = predictions_reshaped.flatten()

# Inverse transform the 'Close' prices using the dummy arrays
y_test_original = scaler_prices.inverse_transform(dummy_array_y_test)[:, -1]
predictions_original = scaler_prices.inverse_transform(dummy_array_predictions)[:, -1]

# Extracting testing dates
test_dates = test_data.index

# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_dates, y=y_test_original, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=test_dates, y=predictions_original, mode='lines', name='Predicted'))
fig.update_layout(title='Actual vs Predicted Stock Prices', xaxis_title='Date', yaxis_title='Stock Price', legend_title='Legend')
fig.show()

Epoch 1/50
2/2 [==============================] - 1s 274ms/step - loss: 0.4814 - val_loss: 0.0128
Epoch 2/50
2/2 [==============================] - 0s 40ms/step - loss: 0.0925 - val_loss: 0.0393
Epoch 3/50
2/2 [==============================] - 0s 35ms/step - loss: 0.1964 - val_loss: 0.0473
Epoch 4/50
2/2 [==============================] - 0s 33ms/step - loss: 0.2786 - val_loss: 0.0292
Epoch 5/50
2/2 [==============================] - 0s 37ms/step - loss: 0.1268 - val_loss: 0.0144
Epoch 6/50
2/2 [==============================] - 0s 38ms/step - loss: 0.0305 - val_loss: 0.0095
Epoch 7/50
2/2 [==============================] - 0s 34ms/step - loss: 0.0200 - val_loss: 0.0102
Epoch 8/50
2/2 [==============================] - 0s 33ms/step - loss: 0.0506 - val_loss: 0.0108
Epoch 9/50
2/2 [==============================] - 0s 38ms/step - loss: 0.0843 - val_loss: 0.0081
Epoch 10/50
2/2 [==============================] - 0s 34ms/step - loss: 0.0320 - val_loss: 0.0075
Epoch 11/50
2/2 [===========

1/1 [==============================] - 0s 60ms/step
MSE: 0.0172397240371582, MAE: 0.11410148700585848, R^2: 0.8526703134786497


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 833 (3.25 KB)
Trainable params: 833 (3.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
